Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms


Define Data Transformations

In [2]:
# Define transformations for the dataset
data_transforms = transforms.Compose([
    transforms.Resize((32, 32)),  # CIFAR100 images are already 32x32
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


Load CIFAR100

In [3]:
# Define dataset root directory
dataset_root = './data'

# Load CIFAR100 dataset
trainset = datasets.CIFAR100(root=dataset_root, train=True, transform=data_transforms, download=True)
testset = datasets.CIFAR100(root=dataset_root, train=False, transform=data_transforms, download=True)


100%|██████████| 169M/169M [00:13<00:00, 13.0MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


Create Data Loaders

In [4]:
# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


Custom CNN Architecture

In [5]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # Conv Layer 1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max Pool Layer 1

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Conv Layer 2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max Pool Layer 2

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # Conv Layer 3
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 512),  # Fully Connected Layer
            nn.ReLU(),
            nn.Linear(512, num_classes)  # Output Layer
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Instantiate the model
model = CustomCNN(num_classes=100)


Set Device and Hyperparams

In [6]:
# Move model to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function, optimizer, and learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


Train Model

In [7]:
# Training loop
num_epochs = 30
best_acc = 0.0
best_model_wts = model.state_dict()

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    model.train()
    running_loss = 0.0
    running_corrects = 0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(trainset)
    epoch_acc = running_corrects.double() / len(trainset)
    print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = model.state_dict()

torch.save(best_model_wts, 'best_custom_model.pth')


Epoch 1/30
----------
Loss: 3.7336 Acc: 0.1333
Epoch 2/30
----------
Loss: 2.8726 Acc: 0.2838
Epoch 3/30
----------
Loss: 2.4029 Acc: 0.3824
Epoch 4/30
----------
Loss: 2.0304 Acc: 0.4621
Epoch 5/30
----------
Loss: 1.7153 Acc: 0.5364
Epoch 6/30
----------
Loss: 1.4144 Acc: 0.6074
Epoch 7/30
----------
Loss: 1.1319 Acc: 0.6828
Epoch 8/30
----------
Loss: 0.8812 Acc: 0.7490
Epoch 9/30
----------
Loss: 0.6703 Acc: 0.8090
Epoch 10/30
----------
Loss: 0.5096 Acc: 0.8514
Epoch 11/30
----------
Loss: 0.4008 Acc: 0.8815
Epoch 12/30
----------
Loss: 0.3066 Acc: 0.9105
Epoch 13/30
----------
Loss: 0.2525 Acc: 0.9255
Epoch 14/30
----------
Loss: 0.2058 Acc: 0.9384
Epoch 15/30
----------
Loss: 0.1818 Acc: 0.9460
Epoch 16/30
----------
Loss: 0.1629 Acc: 0.9504
Epoch 17/30
----------
Loss: 0.1565 Acc: 0.9521
Epoch 18/30
----------
Loss: 0.1457 Acc: 0.9551
Epoch 19/30
----------
Loss: 0.1323 Acc: 0.9590
Epoch 20/30
----------
Loss: 0.1411 Acc: 0.9566
Epoch 21/30
----------
Loss: 0.1178 Acc: 0.9643
E

Test Model

In [8]:
# Load the best model weights
model.load_state_dict(torch.load('best_custom_model.pth'))
model.eval()

running_corrects = 0

for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(testset)
print(f'Test Accuracy: {test_acc:.4f}')


<ipython-input-8-4af2c7bdef62>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_custom_model.pth'))


Test Accuracy: 0.4569
